# Beampattern Notebook

Concepts to communicate:

1) Beampattern
2) Main Lobe
3) Sidelobes
4) Beam width
5) Source Level
6) Near-field vs far-field

A **transducer** is a device that takes an electrical signal and transforms it into mechanical motion. The audio speakers in computers, phones, or headphones are probably the transducers we interact with and use on a daily basis. These transducers use an electrical signal to cause the speaker surface to oscillate which then generates sound. The acoustic source in a sonar system does the same thing. A transducer can also take mechanical motion and transform it into an electrical signal (microphones in air and hydrophones in the water), but this section will focus only on the properties of acoustic transducers that are used to generate sound.

## Circular transducer

The most common man-made sound sources used in underwater acoustics are planer transducers which have flat surfaces which oscillate to produce sound. This is in part due to the fact that a flat transducer is relatively easy to manufacturer, but more importantly, the transducer transmits a majority of the sound in the direction perpendicular to the vibrating plane.  This is a useful property in many sonar systems and this directivity of the transmitted sound can help determine the location of an object (fish, bubble, submarine, etc.)

Note that we said the transducer directs a "majority" of the sound in one direction and not "all" of the sound. In fact, the transducer sends different levels of sound in many directions and the directional dependence of the sound is refered to as the transducer's **beam pattern**. To help understand the concept of a beam pattern and it's properties, we will focus initially on one particular, common type of tranducer: the circular piston. This type of transducer is often a cylindrical piece of piezoelectric material to which an electrical signal is applied such that it expands and contracts along the axis of the cylinder. The causes sound to radiate from the circular face of the transducer. The structure of the sound field from this type of transducer can be fairly complicated and there is not a simple, mathematical expression for the direction and strength of the transmitted sound. There are, however, approximate mathematical descriptions that can be accurately describe the transmitted sound under certain conditions. One of the most useful of these is to assume that the transducer an be approximated as a baffled circular piston. By "baffled," we mean that the face of the transducer is surrounded by an infinite, rigid surface. This simplifies the mathematics and allows us to calculate strength and direction of the transmitted field.

In the following, the face of the transducer will be in the x-y plane and the z-axis is perpendicular to the face. The radius of the tranducer is ***a*** and the field will be described in two ways; either a function of the distance, ***r***, from the center of the face and the angle, ***$\theta$***, between the direction of ***r*** and the -axis or as a function of distance from the plane of the transducer (range) and the perpendicular direction (cross-range.)

![Piston](quick_piston.png)


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1  # Bessel function of the first kind
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd

In [3]:
# Constants
rho0 = 1
c = 1500
r = 10

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='Aperture Radius (m)')

def update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale):

    omega = 2 * np.pi * f * 1000
    k = omega / c
    theta = np.arange(-90, 90.1, 0.1)
    u = k * a * np.sin(theta * np.pi / 180)

    # Projection Intensity calculation
    projInt = np.abs((rho0 * c * k * a * a / (2 * r)) * 
                     (2 * j1(u) / u)) ** 2

    # Handle division by zero at theta = 0
    ind = np.where(theta == 0)[0]
    axisInt = np.abs(rho0 * c * k * a * a / (2 * r)) ** 2
    projInt[ind] = axisInt

    plt.close('all')  # Close any existing plots

    if plotType == 'Cartesian':
        if plotScale == "Decibels":
            plt.plot(theta, 10 * np.log10(projInt / axisInt), linewidth=2)
            plt.ylim([-40, 0])
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level (dB)', fontsize=14)
            plt.grid(True)
        else:
            plt.plot(theta, projInt / axisInt, linewidth=2)
            plt.ylim(10 ** (np.array([-40, 0]) / 10))
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level', fontsize=14)
            plt.grid(True)
    
    elif plotType == 'Polar':
        if plotScale == "Decibels":
            plt.polar(theta * np.pi / 180, 10 * np.log10(projInt / axisInt), linewidth=2)
            ax = plt.gca()
            ax.set_ylim([-40, 0])
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (dB)', fontsize=14)
        else:
            plt.polar(theta * np.pi / 180, projInt / axisInt, linewidth=2)
            ax = plt.gca()
            ax.set_ylim(10 ** (np.array([-40, 0]) / 10))
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (Linear Scale)', fontsize=14)

    # Compute beam widths
    ind3dB = np.where(10 * np.log10(projInt / axisInt) >= -3)[0]
    ind6dB = np.where(10 * np.log10(projInt / axisInt) >= -6)[0]
    ind10dB = np.where(10 * np.log10(projInt / axisInt) >= -10)[0]

    bw3dB = np.abs(theta[ind3dB[0]])
    bw6dB = np.abs(theta[ind6dB[0]])
    bw10dB = np.abs(theta[ind10dB[0]])

    # Define labels and values
    bwLabels = ['down 3 dB', 'down 6 dB', 'down 10 dB']
    bwValues = [bw3dB, bw6dB, bw10dB]

    # Create a DataFrame
    df = pd.DataFrame({
        'Beam Width (degrees)': bwValues,
        'Definition': bwLabels
    })

    # Print the DataFrame
    print(df)

# Create widgets for plot type and scale
plot_type_widget = widgets.Dropdown(
    options=['Cartesian', 'Polar'],
    value='Polar',
    description='Plot Type:'
)

plot_scale_widget = widgets.Dropdown(
    options=['Decibels', 'Linear'],
    value='Decibels',
    description='Scale:'
)

# Use interact to link widgets and plotting function
interact(
    update_far_field_intensity_cartesian_polar_plot,
    f=f_slider,
    a=a_slider,
    plotType=plot_type_widget,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=1, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=0.5,…

<function __main__.update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale)>

In [67]:
# Constants
rho0 = 1
c = 1500

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description='Aperture Radius (m)')

def far_field_intensity_x_y_plane_calculation(f, a, r, cr):
    omega = 2 * np.pi * f * 1000
    k = omega / c

    # Calculations
    SL_dB = 180
    SL_lin = 10**(SL_dB / 20)
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    x = np.linspace(-cr, cr, 150)
    y = np.linspace(0, r, 100)
    X, Y = np.meshgrid(x, y)

    thetaXY = np.arctan2(X, Y)
    rXY = np.sqrt(X**2 + Y**2)
    u1 = k * a * np.sin(thetaXY)
    u1[u1 == 0] = np.nan  # Avoid division by zero

    projIntXY = np.abs((rho0 * c * k * U0 * a * a / (2 * rXY)) *
                    (2 * j1(u1) / u1))**2

    return x, y, SL_dB, projIntXY

def update_far_field_intensity_x_y_plane_plot(f, a, plotScale):
    # Calculate far field intensity on x-y plane
    x, y, SL_dB, projIntXY = far_field_intensity_x_y_plane_calculation(f, a, r=10, cr=10)

    # Close any existing plots
    plt.close('all')

    # Plotting
    plt.figure(1)
    if plotScale == 'Decibels':
        plt.imshow(10 * np.log10(projIntXY), extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (dB re 1μPa)')
        plt.clim(SL_dB - 90, SL_dB)
    else:
        plt.imshow(projIntXY / 1e6, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (μPa²)')
        plt.clim(10**((SL_dB - 90) / 10), SL_dB)
        
    plt.xlabel('Cross Range (m)')
    plt.ylabel('Range (m)')
    plt.gca().set_aspect('auto')
    plt.show()

# Use interact to link widgets and plotting function
interact(
    update_far_field_intensity_x_y_plane_plot,
    f=f_slider,
    a=a_slider,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=1, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=0.5,…

<function __main__.update_far_field_intensity_x_y_plane_plot(f, a, plotScale)>

In [158]:
# Constants
rho0 = 1
c = 1500

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.5, description='Aperture Radius (m)')
r_slider = widgets.IntSlider(min=1, max=10, step=1, value=10, description='Range (m)')
cr_slider = widgets.IntSlider(min=1, max=10, step=1, value=10, description='Cross Range (m)')

def update_full_field_intensity_x_y_plane_plot(f, a, r, cr):
    omega = 2 * np.pi * f * 1000
    global k
    k = omega / c

    # Calculations
    SL_dB = 180
    SL_lin = 10**(SL_dB / 20)
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    # Create mesh grids
    x1 = np.linspace(-cr, 0, 50)
    x1 = np.concatenate((x1, -x1[-2::-1]))
    y1 = np.linspace(0, r, 200)
    X1, Y1 = np.meshgrid(x1, y1)

    # Calculate thetaXY1, rXY1, and u11
    thetaXY1 = np.arctan2(X1, Y1)
    rXY1 = np.sqrt(X1**2 + Y1**2)
    rXY1[rXY1 == 0.0] = 1e-6
    u11 = k * a * np.sin(thetaXY1)
    u11[u11 == 0.0] = np.nan
    projIntXY1 = np.abs((rho0 * c * k * U0 * a**2 / (2 * rXY1)) * (2 * j1(u11) / u11))**2
    indX1 = np.argmin(np.abs(x1))
    projIntXY1[:, indX1] = np.abs(rho0 * c * U0 * k * a**2 / (2 * rXY1[:, indX1]))**2

    # Create sigmaP and phiP
    global sigmaP
    global phiP
    global dsigmaP
    global dphiP
    sigmaP = np.linspace(0, a, 200)
    phiP = np.linspace(0, 2 * np.pi, 200)
    dsigmaP = np.diff(sigmaP[:2])
    dphiP = np.diff(phiP[:2])

    # Calculate intValXY1
    global intValXY1
    intValXY1 = np.zeros_like(X1, dtype=complex)
    for jX in range(len(x1)):
        for jY in range(len(y1)):
            rPXY1 = np.sqrt(
                rXY1[jY, jX]**2 + sigmaP**2 + 2*rXY1[jY, jX]*sigmaP*np.sin(thetaXY1[jY, jX])*(np.cos(thetaXY1[jY, jX])*np.cos(phiP.T) + np.sin(thetaXY1[jY, jX])*np.sin(phiP.T))
            )
            intValXY1[jY, jX] = np.sum(np.sum(sigmaP * dsigmaP * dphiP * np.exp(-1j * k * rPXY1) / rPXY1))
    global nfprojIntXY
    # Calculate nfprojIntXY
    nfprojIntXY = np.abs((1j * rho0 * c * U0 * k / (2 * np.pi) * intValXY1)**2)

    # Create subplots
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Left subplot
    im1 = ax1.imshow(10 * np.log10(projIntXY1), extent=(x1.min(), x1.max(), y1.min(), y1.max()), origin='lower')
    ax1.set_xlabel('Cross Range (m)')
    ax1.set_ylabel('Range (m)')
    plt.colorbar(im1, ax=ax1, label='Intensity (dB re 1μPa)')
    im1.set_clim(SL_dB - 90, SL_dB)
    ax1.set_aspect('auto')

    # Right subplot
    im2 = ax2.imshow(10 * np.log10(nfprojIntXY), extent=(x1.min(), x1.max(), y1.min(), y1.max()), origin='lower')
    ax2.set_xlabel('Cross Range (m)')
    ax2.set_ylabel('Range (m)')
    plt.colorbar(im2, ax=ax2, label='Intensity (dB re 1μPa)')
    im2.set_clim(SL_dB - 90, SL_dB)
    ax2.set_aspect('auto')

    plt.tight_layout()
    plt.show()

# Use interact to link widgets and plotting function
interact(
    update_full_field_intensity_x_y_plane_plot,
    f=f_slider,
    a=a_slider,
    r=r_slider,
    cr=cr_slider,
)

interactive(children=(IntSlider(value=1, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=0.5,…

<function __main__.update_full_field_intensity_x_y_plane_plot(f, a, r, cr)>

In [172]:
np.exp(-1j * k * rPXY1) / rPXY1

array([-0.0636151 -0.03087262j, -0.06377585-0.03052416j,
       -0.06394393-0.03015441j, -0.0641185 -0.0297643j ,
       -0.06429869-0.02935483j, -0.06448367-0.02892703j,
       -0.06467256-0.02848199j, -0.06486452-0.02802084j,
       -0.0650587 -0.02754474j, -0.06525429-0.02705493j,
       -0.06545046-0.02655267j, -0.06564644-0.02603926j,
       -0.06584147-0.02551606j, -0.06603484-0.02498444j,
       -0.06622585-0.02444584j, -0.06641387-0.0239017j ,
       -0.06659828-0.02335351j, -0.06677853-0.0228028j ,
       -0.06695411-0.02225109j, -0.06712455-0.02169996j,
       -0.06728943-0.02115098j, -0.0674484 -0.02060574j,
       -0.06760112-0.02006586j, -0.06774735-0.01953294j,
       -0.06788685-0.01900859j, -0.06801947-0.01849443j,
       -0.06814506-0.01799208j, -0.06826355-0.01750312j,
       -0.06837488-0.01702914j, -0.06847907-0.01657173j,
       -0.06857612-0.01613242j, -0.06866609-0.01571275j,
       -0.06874905-0.01531422j, -0.06882512-0.0149383j ,
       -0.06889439-0.01458642j,

In [179]:
sigmaP * dsigmaP * dphiP * np.exp(-1j * k * rPXY1)


array([ 0.00000000e+00-0.00000000e+00j, -1.79792559e-07-8.60516452e-08j,
       -3.60567738e-07-1.70035012e-07j, -5.42383432e-07-2.51778595e-07j,
       -7.25287761e-07-3.31121822e-07j, -9.09318984e-07-4.07915715e-07j,
       -1.09450550e-06-4.82023551e-07j, -1.28086593e-06-5.53321526e-07j,
       -1.46840929e-06-6.21699402e-07j, -1.65713525e-06-6.87061142e-07j,
       -1.84703444e-06-7.49325523e-07j, -2.03808892e-06-8.08426729e-07j,
       -2.23027257e-06-8.64314902e-07j, -2.42355167e-06-9.16956664e-07j,
       -2.61788550e-06-9.66335597e-07j, -2.81322694e-06-1.01245266e-06j,
       -3.00952317e-06-1.05532659e-06j, -3.20671637e-06-1.09499420e-06j,
       -3.40474443e-06-1.13151062e-06j, -3.60354169e-06-1.16494955e-06j,
       -3.80303967e-06-1.19540335e-06j, -4.00316781e-06-1.22298310e-06j,
       -4.20385414e-06-1.24781862e-06j, -4.40502598e-06-1.27005835e-06j,
       -4.60661059e-06-1.28986927e-06j, -4.80853571e-06-1.30743661e-06j,
       -5.01073010e-06-1.32296362e-06j, -5.21312403